## Installs for new environments

In [4]:
# import sys
# !{sys.executable} -m pip install tensorflow
# !{sys.executable} -m pip install matplotlib
# !{sys.executable} -m pip install numpy
# !{sys.executable} -m pip install pillow
# !{sys.executable} -m pip install scipy

In [1]:
%load_ext tensorboard

## Imports

In [2]:
import tensorflow as tf
import matplotlib.pyplot as plt
import os
import numpy as np
from tensorflow.keras.preprocessing.image import ImageDataGenerator, img_to_array, load_img
from tensorflow.keras.preprocessing import image
from tensorflow.keras.optimizers import RMSprop
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense
import math

import datetime


## Config

In [3]:
image_dim = 256 # eventual width/height of images
input_shape = (image_dim, image_dim, 3) #(width, height, RGB)

epochs = 15
batch_size = 32
learning_rate = .001

train_dir = 'dataset/train'
validate_dir = 'dataset/validate'

save_dir = 'models/reinforced_full'
file_name_base = 'reinforced_full'

base_log_dir = '/logs'

## Reinforcement Definition

In [4]:
train = ImageDataGenerator(rescale=1./255,
    shear_range=0.2,
    zoom_range=0.2,
    rotation_range=20,
    horizontal_flip=True) # Reinforcements for training dataset

validate = ImageDataGenerator(rescale=1./255) # Reinforcements for validation dataset

## Dataset Loading

In [5]:
#loads the datasets
train_dataset = train.flow_from_directory(train_dir, target_size=(image_dim, image_dim), class_mode='categorical') 
validate_dataset = train.flow_from_directory(validate_dir, target_size=(image_dim, image_dim), class_mode='categorical')

# tf.keras.preprocessing.image_dataset_from_directory - this can be used for no reinforcement

n_train = train_dataset.samples # number of images in training dataset
n_validate = validate_dataset.samples # number of images in validation dataset


Found 8000 images belonging to 10 classes.
Found 2000 images belonging to 10 classes.


## CNN Design

In [6]:
class_keys = list(train_dataset.class_indices.keys())
n_classes = len(class_keys)

# defines the layers of the cnn
model = tf.keras.models.Sequential([
    Conv2D(16, (3, 3), activation='relu', padding='same', input_shape=input_shape),
    Conv2D(16, (3, 3), activation='relu', padding='same'),
    MaxPooling2D(pool_size=(2,2), strides=None, padding='valid'),
    Conv2D(32, (3, 3), activation='relu', padding='same'),
    Conv2D(32, (3, 3), activation='relu', padding='same'),
    MaxPooling2D(pool_size=(2,2), strides=None, padding='valid'),
    Conv2D(64, (3, 3), activation='relu', padding='same'),
    Conv2D(64, (3, 3), activation='relu', padding='same'),
    MaxPooling2D(pool_size=(2,2), strides=None, padding='valid'),
    Conv2D(128, (3, 3), activation='relu', padding='same'),
    Conv2D(128, (3, 3), activation='relu', padding='same'),
    MaxPooling2D(pool_size=(2,2), strides=None, padding='valid'),
    Flatten(),
    Dense(256, activation='relu'),
    Dense(n_classes, activation='softmax')
])

## Compilation

In [7]:
log_dir = base_log_dir + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)

model.compile(optimizer=tf.keras.optimizers.SGD(learning_rate=learning_rate, momentum=0.9),
              loss='categorical_crossentropy',
              metrics=['accuracy'])

## Evaluate and Save the initial model

In [8]:
model.evaluate(validate_dataset, batch_size=256)
model.save(save_dir + '/' + file_name_base + '_0')

63/63 [==============================] - 76s 1s/step - loss: 2.3031 - accuracy: 0.0985
INFO:tensorflow:Assets written to: models/reinforced_full/reinforced_full_0\assets


## Training

In [11]:
model.fit(train_dataset,
                    steps_per_epoch=math.floor(n_train/batch_size),
                    validation_data=validate_dataset,
                    validation_steps=math.floor(n_validate/batch_size),
                    epochs=epochs,
                    callbacks=[tensorboard])

Epoch 1/15
250/250 [==============================] - 887s 4s/step - loss: 1.3661 - accuracy: 0.5288 - val_loss: 1.5615 - val_accuracy: 0.4677
Epoch 2/15
250/250 [==============================] - 754s 3s/step - loss: 1.3215 - accuracy: 0.5471 - val_loss: 1.4628 - val_accuracy: 0.4919
Epoch 3/15
250/250 [==============================] - 746s 3s/step - loss: 1.2761 - accuracy: 0.5686 - val_loss: 1.4298 - val_accuracy: 0.5106
Epoch 4/15
250/250 [==============================] - 747s 3s/step - loss: 1.2335 - accuracy: 0.5781 - val_loss: 1.4329 - val_accuracy: 0.5181
Epoch 5/15
250/250 [==============================] - 746s 3s/step - loss: 1.2089 - accuracy: 0.5893 - val_loss: 1.3874 - val_accuracy: 0.5227
Epoch 6/15
191/250 [=====================>........] - ETA: 2:43 - loss: 1.1663 - accuracy: 0.6113

## Save Trained Model

In [10]:
model.save(save_dir + '/' + file_name_base + '_15')

INFO:tensorflow:Assets written to: models/reinforced_full/reinforced_full_15\assets
